In [ ]:
from google.colab import drive
drive.mount('/content/drive/')


import os
os.chdir("drive/My Drive/Advanced NLP/Exam")

import pickle
import random
import time
from collections import Counter, defaultdict

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import torch
import torch.nn as nn
from torch import optim

plt.switch_backend('agg')
import numpy as np
from tqdm import tqdm

from models import EncoderGRU, AttnDecoderGRU, EncoderLSTM, DecoderLSTM, AttnDecoderLSTM
from utils import Lang, tensorsFromPair, timeSince, showPlot


In [10]:

SOS_token = 0
EOS_token = 1

command_le = Lang('command')
action_le = Lang('action')

class Format:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.index2word = {SOS_token: "SOS", EOS_token: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence:
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.index2word[self.n_words] = word
            self.n_words += 1
            
def readFile(filename):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('SCAN-master/%s.txt' % filename, encoding='utf-8').read().strip().split('\n')

    pairs = [s[4:].split(' OUT: ') for s in lines]

    input_lang = Format("input")
    output_lang = Format("output")

    return input_lang, output_lang, pairs

def prepareData(filename):
    input_lang, output_lang, pairs = readFile(filename)
    print("Read %s sentence pairs" % len(pairs))

    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence]# .split(' ')


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

def tensorsFromPair(pair, input_lang, output_lang):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset_path = '../SCAN-master'
task_name = 'add_prim'  # or 'length'
primitive = 'jump'

train_file_name = '{}_split/tasks_{}_{}.txt'.format(task_name, 'train', 'addprim'+'_'+primitive)
test_file_name = '{}_split/tasks_{}_{}.txt'.format(task_name, 'test', 'addprim'+'_'+primitive)
train_file_path = os.path.join(dataset_path, train_file_name)
test_file_path = os.path.join(dataset_path, test_file_name)

# train_file_path, test_file_path

SOS_token = 0
EOS_token = 1

command_le = Lang('command')
action_le = Lang('action')


def dataloader(path):
    with open(path, 'r') as f:
        dataset = f.readlines()

    def preprocess_data(line):
        line = line.strip().split()
        split_index = line.index('OUT:')
        inp = line[1: split_index]
        outp = line[split_index + 1:]
        command_le.addSentence(inp)
        action_le.addSentence(outp)
        return [inp, outp]

    pairs = list(map(preprocess_data, dataset))
    input_commands, output_actions = np.transpose(pairs).tolist()
    return input_commands, output_actions, pairs


commands_train, actions_train, pairs_train = dataloader(train_file_path)
commands_test, actions_test, pairs_test = dataloader(test_file_path)

MAX_LENGTH = max([len(action) for action in actions_test]) + 1

def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence] # .split(' ')


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair, command_le, action_le):
    input_tensor = tensorFromSentence(command_le, pair[0])
    target_tensor = tensorFromSentence(action_le, pair[1])
    return (input_tensor, target_tensor)


def evaluate(encoder, decoder, pair, model='gru', attention=True, max_length=MAX_LENGTH, device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
    with torch.no_grad():
        input_tensor, target_tensor = tensorsFromPair(pair, command_le, action_le)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_hiddens = torch.zeros(input_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            
            if attention:
                if model == 'lstm':
                    encoder_hiddens[ei] += encoder_hidden[0][0,0]
                elif model == 'gru':
                    encoder_hiddens[ei] += encoder_hidden[0,0]


        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, input_length)

        for di in range(max_length):
            if attention:
                decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input, decoder_hidden, encoder_hiddens)
                decoder_attentions[di] = decoder_attention.squeeze().data.to(device)
            else:
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(action_le.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di+1]


def evaluate_search_failure(encoder, decoder, pair, max_length=MAX_LENGTH, attention=False):
    with torch.no_grad():
        input_tensor, target_tensor = tensorsFromPair(pair, command_le, action_le)
        target_output = pair[1]
        input_length = input_tensor.size()[0]
        target_length = target_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_hiddens = torch.zeros(input_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_hiddens[ei] += encoder_output[0,0]

        decoded_words_tf = []
        decoded_words_sg = []
        cumul_prob_tf = torch.tensor(0, dtype=torch.float).to(device)
        cumul_prob_sg = torch.tensor(0, dtype=torch.float).to(device)
        
        # Teacher forcing evaluation
        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden
        
        for di in range(target_length):
            if attention:
                decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input, decoder_hidden, encoder_hiddens)
            else:
                decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            
            topv, topi = decoder_output.data.topk(1)
            cumul_prob_tf += topv.squeeze().item()
            
            if topi.item() == EOS_token:
                break
            else:
                decoded_words_tf.append(action_le.index2word[topi.item()])
                
            decoder_input = target_tensor[di]
        
        # Self generated evaluation
        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden
        
        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input, decoder_hidden, encoder_hiddens)
            
            topv, topi = decoder_output.data.topk(1)
            cumul_prob_sg += topv.squeeze().item()
            
            if topi.item() == EOS_token:
                break
            else:
                decoded_words_sg.append(action_le.index2word[topi.item()])
                
            decoder_input = topi.squeeze()
        
        both_wrong = (decoded_words_tf != target_output) and (decoded_words_sg != target_output)
        return both_wrong, cumul_prob_sg.item(), cumul_prob_tf.item()


def evaluate_model_search_failure(encoder, decoder, test_pairs, attention=False):
    probs_sg = []
    probs_tf = []
    
    for pair in tqdm(test_pairs):
        both_wrong, sg_prob, tf_prob = evaluate_search_failure(encoder, decoder, pair, attention)
        
        if both_wrong:
            probs_sg.append(sg_prob)
            probs_tf.append(tf_prob)
            
    return probs_sg, probs_tf

In [2]:
# Print errors for turn left and jump
# Overall-best model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

input_size = command_le.n_words
output_size = action_le.n_words

best_encoder_lstm_turn = EncoderLSTM(input_size=input_size, hidden_size=200, dropout=0.5, num_layers=2)
best_decoder_lstm_turn = DecoderLSTM(hidden_size=200, output_size=output_size, dropout=0.5, num_layers=2)

best_encoder_lstm_turn.load_state_dict(torch.load(f"encoder_3a2_lstm_False.pt"))
best_decoder_lstm_turn.load_state_dict(torch.load(f"decoder3a2_lstm_False.pt"))

best_encoder_lstm_jump = EncoderLSTM(input_size=input_size, hidden_size=200, dropout=0.5, num_layers=2)
best_decoder_lstm_jump = DecoderLSTM(hidden_size=200, output_size=output_size, dropout=0.5, num_layers=2)

best_encoder_lstm_jump.load_state_dict(torch.load(f"encoder_3b2_lstm_False.pt"))
best_decoder_lstm_jump.load_state_dict(torch.load(f"decoder3b2_lstm_False.pt"))



<All keys matched successfully>

In [ ]:
task = "turn_left"

dataset_path = '../SCAN-master/add_prim_split/'

train_file_name = f'tasks_train_addprim_{task}.txt'
test_file_name = f'tasks_test_addprim_{task}.txt'

train_file_path = os.path.join(dataset_path, train_file_name)
test_file_path = os.path.join(dataset_path, test_file_name)

command_le = Format('input')
action_le = Format('output')

commands_train, actions_train, pairs_train = dataloader(train_file_path)
commands_test, actions_test, pairs_test = dataloader(test_file_path)

total = 0
turn_left = 0
turn_left_thrice = 0 
turn_left_twice = 0

counts = {"turn left":0, "turn left thrice":0}
counts_testset = {}
errors = []

best_encoder_lstm_turn.to(device)
best_decoder_lstm_turn.to(device)

for i in range(len(pairs_test)):
        counts_testset[" ".join(pairs_test[i][0])] = 0
        counts[" ".join(pairs_test[i][0])] = 0
        preds, attentions = evaluate(best_encoder_lstm_turn, best_decoder_lstm_turn, pairs_test[i], model = 'lstm', attention=False)
        preds = preds[:-1]
        target_output = pairs_test[i][1]
        if preds != target_output:
            if " ".join(pairs_test[i][0]).startswith("turn left thrice"):
                counts["turn left thrice"] = counts.get("turn left thrice", 0) + 1
            elif " ".join(pairs_test[i][0]).startswith("turn left"):
                counts["turn left"] = counts.get("turn left", 0) + 1
            errors.append(" ".join(pairs_test[i][0]))
            total+=1
            counts[" ".join(pairs_test[i][0])] = counts.get(" ".join(pairs_test[i][0]), 0) + 1

        counts_testset[" ".join(pairs_test[i][0])] = counts_testset.get(" ".join(pairs_test[i][0]), 0) + 1

probs_sg, probs_tf = evaluate_model_search_failure(best_encoder_lstm_turn, best_decoder_lstm_turn, pairs_test, attention=False)

In [ ]:
task = "jump"

dataset_path = '../SCAN-master/add_prim_split/'

train_file_name = f'tasks_train_addprim_{task}.txt'
test_file_name = f'tasks_test_addprim_{task}.txt'

train_file_path = os.path.join(dataset_path, train_file_name)
test_file_path = os.path.join(dataset_path, test_file_name)

command_le = Format('input')
action_le = Format('output')

commands_train, actions_train, pairs_train = dataloader(train_file_path)
commands_test, actions_test, pairs_test = dataloader(test_file_path)

for i in range(len(pairs_test)):
        preds, attentions = evaluate(best_encoder_lstm_jump.to(device), best_decoder_lstm_jump.to(device), pairs_test[i], model="lstm", attention=False)
        preds = preds[:-1]
        target_output = pairs_test[i][1]
        if preds == target_output:
            print(pairs_test[i][0])



In [14]:
# Testing for search errors
task = "jump"

dataset_path = '../SCAN-master/add_prim_split/'

train_file_name = f'tasks_train_addprim_{task}.txt'
test_file_name = f'tasks_test_addprim_{task}.txt'

train_file_path = os.path.join(dataset_path, train_file_name)
test_file_path = os.path.join(dataset_path, test_file_name)

command_le = Format('input')
action_le = Format('output')

def dataloader(path):
    with open(path, 'r') as f:
        dataset = f.readlines()

    def preprocess_data(line):
        line = line.strip().split()
        split_index = line.index('OUT:')
        inp = line[1: split_index]
        outp = line[split_index + 1:]
        command_le.addSentence(inp)
        action_le.addSentence(outp)
        return [inp, outp]

    pairs = list(map(preprocess_data, dataset))
    input_commands, output_actions = np.transpose(pairs).tolist()
    return input_commands, output_actions, pairs


commands_train, actions_train, pairs_train = dataloader(train_file_path)
commands_test, actions_test, pairs_test = dataloader(test_file_path)

input_size = command_le.n_words
output_size = action_le.n_words

probs_sg, probs_tf = evaluate_model_search_failure(best_encoder_lstm_jump, best_decoder_lstm_jump, pairs_test, attention=False)

100%|██████████| 7706/7706 [01:02<00:00, 123.15it/s]
